This notebook serves to document the progression of optimization found in the queries that search and analyze the movies. It documents the version number, execution time (on the individual's device), and optimization strategies utilized in optimizing the query.

**Movie Search Function**

| **Version** | **Execution Time** | **Optimization Strategies Utilized** |
|----------|----------|----------|
| Preperation | No Recorded Time | First optimization, added indexes across a lot of tables (as stated the logic is used for all types of searches, so indexes were added for more than just genres.) |
| Naive |  .390 seconds | Naive solution, WHERE to filter for the genre, rating, and keyword |
| Optimized | .006 seconds (major improvement) | Final optimized version, improved the query by using cleaner WHERE statements instead of a subquery  |

Indexes

In [ ]:
%%sql

CREATE INDEX idx_movie_tconst 
    ON dim_movie(tconst(12));
CREATE INDEX idx_movie_title 
    ON dim_movie(primaryTitle(100));
CREATE INDEX idx_ratings_tconst 
    ON title_ratings(tconst(12));
CREATE INDEX idx_genre_id 
    ON genre_dt(genreID);
CREATE INDEX idx_tg_genreID 
    ON title_genre_bridge(genreID);
CREATE INDEX idx_tg_tconst 
    ON title_genre_bridge(tconst(12));
CREATE FULLTEXT INDEX idx_movie_fulltext_title 
    ON dim_movie(primaryTitle);
CREATE INDEX idx_genre_bridge_genreID_tconst 
    ON title_genre_bridge(genreID, tconst(12));
CREATE INDEX idx_genre_name 
    ON genre_dt(genreName);
CREATE INDEX idx_tg_genreID_tconst 
    ON title_genre_bridge(genreID, tconst(12));
CREATE INDEX idx_ratings_tconst_avgRating 
    ON title_ratings(tconst(12), averageRating);
CREATE INDEX idx_rating_votes 
    ON title_ratings(averageRating, numVotes)
CREATE INDEX idx_votes_rating 
    ON title_ratings(numVotes, averageRating)

Naive 

In [ ]:
%%sql

SELECT 
    m.tconst,
    m.primaryTitle AS title,
    r.averageRating,
    r.numVotes,
    GROUP_CONCAT(DISTINCT g.genreName) AS genres
FROM dim_movie AS m
JOIN title_ratings AS r ON m.tconst = r.tconst
LEFT JOIN title_genre_bridge AS tg ON m.tconst = tg.tconst
LEFT JOIN genre_dt AS g ON tg.genreID = g.genreID

# with this next section in python as to get the parameters. refer to the application to run
#where_clauses = []
#    params = {}
    
#    if keyword:
        where_clauses.append("m.primaryTitle LIKE :keyword")
        params["keyword"] = f"%{keyword}%"
        
#    if genre:
#        where_clauses.append("""
            m.tconst IN (
                SELECT tg_sub.tconst 
                FROM title_genre_bridge AS tg_sub
                JOIN genre_dt AS g_sub ON tg_sub.genreID = g_sub.genreID
                WHERE g_sub.genreName = :genre
            )
#        """)
        params["genre"] = genre
        
#    if min_rating:
        where_clauses.append("r.averageRating >= :min_r")
        params["min_r"] = min_rating
        
#    if max_rating:
        where_clauses.append("r.averageRating <= :max_r")
        params["max_r"] = max_rating
        
#    if min_votes:
        where_clauses.append("r.numVotes >= :min_v")
        params["min_v"] = min_votes
        
#    if where_clauses:
        query += " WHERE " + " AND ".join(where_clauses)
#end of section

GROUP BY m.tconst, m.primaryTitle, r.averageRating, r.numVotes
ORDER BY r.numVotes DESC;



In [ ]:
#full python codeblock section

def combined_search_unoptimized(keyword=None, genre=None, min_rating=None, max_rating=None, min_votes=None):
    query = """
        SELECT 
            m.tconst,
            m.primaryTitle AS title,
            r.averageRating,
            r.numVotes,
            GROUP_CONCAT(DISTINCT g.genreName) AS genres
        FROM dim_movie AS m
        JOIN title_ratings AS r ON m.tconst = r.tconst
        LEFT JOIN title_genre_bridge AS tg ON m.tconst = tg.tconst
        LEFT JOIN genre_dt AS g ON tg.genreID = g.genreID
    """
    
    where_clauses = []
    params = {}
    
    if keyword:
        where_clauses.append("m.primaryTitle LIKE :keyword")
        params["keyword"] = f"%{keyword}%"
        
    if genre:
        where_clauses.append("""
            m.tconst IN (
                SELECT tg_sub.tconst 
                FROM title_genre_bridge AS tg_sub
                JOIN genre_dt AS g_sub ON tg_sub.genreID = g_sub.genreID
                WHERE g_sub.genreName = :genre
            )
        """)
        params["genre"] = genre
        
    if min_rating:
        where_clauses.append("r.averageRating >= :min_r")
        params["min_r"] = min_rating
        
    if max_rating:
        where_clauses.append("r.averageRating <= :max_r")
        params["max_r"] = max_rating
        
    if min_votes:
        where_clauses.append("r.numVotes >= :min_v")
        params["min_v"] = min_votes
        
    if where_clauses:
        query += " WHERE " + " AND ".join(where_clauses)
        
    query += " GROUP BY m.tconst, m.primaryTitle, r.averageRating, r.numVotes"
    query += " ORDER BY r.numVotes DESC"
    
    with engine.connect() as conn:
        start = time.time()
        df = pd.read_sql(text(query), conn, params=params)
        end = time.time()
    
    print(f"Unoptimized COMBINED search executed in {end - start:.3f} seconds")
    print(f"Returned {len(df)} rows")
    return df

Final Optimization With Indexes

In [ ]:
%%sql

#def combined_search_optimized(keyword=None, genre=None, min_rating=None, max_rating=None, min_votes=None):
    select_clause = "SELECT m.tconst, m.primaryTitle AS title, r.averageRating, r.numVotes"
    from_clause = "FROM dim_movie AS m JOIN title_ratings AS r ON m.tconst = r.tconst"
#    join_clauses = []
#    where_clauses = []
#    params = {}
    
#    if keyword:
        where_clauses.append("MATCH(m.primaryTitle) AGAINST(:keyword IN NATURAL LANGUAGE MODE)")
        params["keyword"] = keyword
        
#    if genre:
        join_clauses.append("JOIN title_genre_bridge AS tg ON m.tconst = tg.tconst")
        join_clauses.append("JOIN genre_dt AS g ON tg.genreID = g.genreID")
        where_clauses.append("g.genreName = :genre")
        params["genre"] = genre
        
#    if min_rating:
        where_clauses.append("r.averageRating >= :min_r")
        params["min_r"] = min_rating
        
#    if max_rating:
        where_clauses.append("r.averageRating <= :max_r")
        params["max_r"] = max_rating
        
#    if min_votes:
        where_clauses.append("r.numVotes >= :min_v")
        params["min_v"] = min_votes
        
    query = select_clause + " " + from_clause
    
#    if join_clauses:
        query += " " + " ".join(join_clauses)
        
#    if where_clauses:
        query += " WHERE " + " AND ".join(where_clauses)
        
    query += " ORDER BY r.numVotes DESC"

In [ ]:
#full python codeblock

def combined_search_optimized(keyword=None, genre=None, min_rating=None, max_rating=None, min_votes=None):
    select_clause = "SELECT m.tconst, m.primaryTitle AS title, r.averageRating, r.numVotes"
    from_clause = "FROM dim_movie AS m JOIN title_ratings AS r ON m.tconst = r.tconst"
    join_clauses = []
    where_clauses = []
    params = {}
    
    if keyword:
        where_clauses.append("MATCH(m.primaryTitle) AGAINST(:keyword IN NATURAL LANGUAGE MODE)")
        params["keyword"] = keyword
        
    if genre:
        join_clauses.append("JOIN title_genre_bridge AS tg ON m.tconst = tg.tconst")
        join_clauses.append("JOIN genre_dt AS g ON tg.genreID = g.genreID")
        where_clauses.append("g.genreName = :genre")
        params["genre"] = genre
        
    if min_rating:
        where_clauses.append("r.averageRating >= :min_r")
        params["min_r"] = min_rating
        
    if max_rating:
        where_clauses.append("r.averageRating <= :max_r")
        params["max_r"] = max_rating
        
    if min_votes:
        where_clauses.append("r.numVotes >= :min_v")
        params["min_v"] = min_votes
        
    query = select_clause + " " + from_clause
    
    if join_clauses:
        query += " " + " ".join(join_clauses)
        
    if where_clauses:
        query += " WHERE " + " AND ".join(where_clauses)
        
    query += " ORDER BY r.numVotes DESC"
    
    with engine.connect() as conn:
        start = time.time()
        df = pd.read_sql(text(query), conn, params=params)
        end = time.time()
    
    print(f"Optimized COMBINED search executed in {end - start:.3f} seconds")
    print(f"Returned {len(df)} rows")
    return df

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

**Cosine Similarity**

This only accounts for the actual query, and not the preperation/setup for the cosine similarity. The parameter given is The Matrix (1999) or 'tt0133093'

| **Version** | **Execution Time** | **Optimization Strategies Utilized** |
|----------|----------|----------|
| Preperation | No Recorded Time | First optimization, added indexes across a lot of tables (as stated the logic is used for all types of searches, so indexes were added for more than just genres.) |
| Naive |  23.864 seconds | Naive solution, unoptimized because the calculations done take a long time |
| Optimized | 1.599 seconds (major improvement) | Final optimized version, improved the query by using a materialized table to only require the calculations once |

Cosine Similarity Setup

In [ ]:
# Cosine Similarity Setup 
print("Fetching normalization stats and genre list...")
stats_query = """
    SELECT 
        MIN(averageRating) AS min_rating, MAX(averageRating) AS max_rating,
        MIN(numVotes) AS min_votes, MAX(numVotes) AS max_votes,
        MIN(runtimeMinutes) AS min_runtime, MAX(runtimeMinutes) AS max_runtime
    FROM title_ft
    WHERE numVotes > 1000; 
"""
with engine.connect() as conn:
    stats = pd.read_sql(stats_query, conn).iloc[0]

print(f"Stats loaded: {stats['min_rating']} - {stats['max_rating']} Rating")

genres_query = "SELECT genreID, genreName FROM genre_dt;"
with engine.connect() as conn:
    genres_df = pd.read_sql(genres_query, conn)

print(f"Loaded {len(genres_df)} genres.")

numerical_features = [
    ("norm_rating", f"((t.averageRating - {stats['min_rating']}) / ({stats['max_rating']} - {stats['min_rating']}))"),
    ("norm_votes", f"((t.numVotes - {stats['min_votes']}) / ({stats['max_votes']} - {stats['min_votes']}))"),
    ("norm_runtime", f"((t.runtimeMinutes - {stats['min_runtime']}) / ({stats['max_runtime']} - {stats['min_runtime']}))")
]

genre_features = []
for _, row in genres_df.iterrows():
    col_name = f"genre_{row['genreName'].replace('-', '_').replace(' ', '_')}"
    sql_case = f"MAX(CASE WHEN g.genreID = {row['genreID']} THEN 1 ELSE 0 END) AS {col_name}"
    genre_features.append((col_name, sql_case))

all_features = numerical_features + genre_features
print(f"Total vector dimensions: {len(all_features)}")

# create featur vector view
numerical_selects = ",\n".join([f"    {sql} AS {name}" for name, sql in numerical_features]) # Numerical feature selections
genre_selects = ",\n".join([f"    {sql}" for name, sql in genre_features]) # Genre feature selections

create_view_query = f"""
CREATE OR REPLACE VIEW movie_feature_vector AS
SELECT 
    t.tconst,
    t.primaryTitle,
{numerical_selects},
{genre_selects}
FROM 
    title_ft t
CROSS JOIN (
    SELECT 
        MIN(averageRating) AS min_rating, MAX(averageRating) AS max_rating,
        MIN(numVotes) AS min_votes, MAX(numVotes) AS max_votes,
        MIN(runtimeMinutes) AS min_runtime, MAX(runtimeMinutes) AS max_runtime
    FROM title_ft
    WHERE numVotes > 1000
) AS stats
LEFT JOIN 
    title_genre_bridge g ON t.tconst = g.tconst
WHERE 
    t.numVotes > 1000 AND t.runtimeMinutes IS NOT NULL
GROUP BY 
    t.tconst, t.primaryTitle, 
    stats.min_rating, stats.max_rating, 
    stats.min_votes, stats.max_votes, 
    stats.min_runtime, stats.max_runtime;
"""

# print(create_view_query) # Uncomment to debugthe full query
print("Creating/Replacing movie_feature_vector VIEW...")

with engine.connect() as conn:
    conn.execute(text("DROP VIEW IF EXISTS movie_feature_vector;"))
    conn.execute(text(create_view_query))

print("View 'movie_feature_vector' created successfully.")

Indexes

In [ ]:
%%sql

CREATE INDEX idx_movie_tconst 
    ON dim_movie(tconst(12));
CREATE INDEX idx_movie_title 
    ON dim_movie(primaryTitle(100));
CREATE INDEX idx_ratings_tconst 
    ON title_ratings(tconst(12));
CREATE INDEX idx_genre_id 
    ON genre_dt(genreID);
CREATE INDEX idx_tg_genreID 
    ON title_genre_bridge(genreID);
CREATE INDEX idx_tg_tconst 
    ON title_genre_bridge(tconst(12));
CREATE FULLTEXT INDEX idx_movie_fulltext_title 
    ON dim_movie(primaryTitle);
CREATE INDEX idx_genre_bridge_genreID_tconst 
    ON title_genre_bridge(genreID, tconst(12));
CREATE INDEX idx_genre_name 
    ON genre_dt(genreName);
CREATE INDEX idx_tg_genreID_tconst 
    ON title_genre_bridge(genreID, tconst(12));
CREATE INDEX idx_ratings_tconst_avgRating 
    ON title_ratings(tconst(12), averageRating);
CREATE INDEX idx_rating_votes 
    ON title_ratings(averageRating, numVotes)
CREATE INDEX idx_votes_rating 
    ON title_ratings(numVotes, averageRating)

Naive 

In [ ]:
%%sql

WITH TargetMovie AS (
    SELECT * FROM movie_feature_vector WHERE tconst = :tconst
)
SELECT 
    B.tconst,
    B.primaryTitle,
    ({dot_product_sql}) AS dot_product,
    ({magnitude_A_sql}) AS mag_A,
    ({magnitude_B_sql}) AS mag_B,
    CASE 
        WHEN ({magnitude_A_sql}) = 0 OR ({magnitude_B_sql}) = 0 THEN 0
        ELSE ({dot_product_sql}) / (({magnitude_A_sql}) * ({magnitude_B_sql}))
    END AS cosine_similarity
FROM 
    TargetMovie A
CROSS JOIN 
    movie_feature_vector B
WHERE 
    A.tconst != B.tconst
ORDER BY 
    cosine_similarity DESC;


Final Optimization With Materialization

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS movie_feature_vector_materialized AS
SELECT * FROM movie_feature_vector;

CREATE INDEX idx_mat_vec_tconst ON movie_feature_vector_materialized(tconst(12));

WITH TargetMovie AS (
        SELECT * FROM movie_feature_vector_materialized WHERE tconst = :tconst
)
SELECT 
        B.tconst,
        B.primaryTitle,
        CASE 
        WHEN ({magnitude_A_sql}) = 0 OR ({magnitude_B_sql}) = 0 THEN 0
        ELSE ({dot_product_sql}) / (({magnitude_A_sql}) * ({magnitude_B_sql}))
        END AS cosine_similarity
FROM 
        TargetMovie A
CROSS JOIN 
        movie_feature_vector_materialized B 
WHERE 
        A.tconst != B.tconst
ORDER BY 
        cosine_similarity DESC;

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

**Movie Search Function**

| **Version** | **Execution Time** | **Optimization Strategies Utilized** |
|----------|----------|----------|  
| Naive | 38.734 seconds  | Naive solution, WHERE to filter for null data and genre, aggregate functions to build tables like GenreStats and MovieRankings |
| Optimized | 19.140 seconds (major improvement) | Final optimized version, indexes were added on (averageRating) and (numVotes)  |

Naive 

In [ ]:
%%sql

WITH GenreMovies AS (
        SELECT 
            ft.tconst, ft.averageRating, ft.numVotes
        FROM 
            title_ft AS ft
        JOIN 
            title_genre_bridge AS tgb ON ft.tconst = tgb.tconst
        JOIN 
            genre_dt AS g ON tgb.genreID = g.genreID
        WHERE 
            g.genreName = %s AND ft.numVotes IS NOT NULL
    ),
    GenreStats AS (
        SELECT 
            AVG(averageRating) AS rating_mean, STDDEV(averageRating) AS rating_std,
            AVG(numVotes) AS votes_mean, STDDEV(numVotes) AS votes_std
        FROM GenreMovies
    ),
    MovieRankings AS (
        SELECT
            gm.tconst, gm.averageRating, gm.numVotes,
            (gm.averageRating - gs.rating_mean) / gs.rating_std AS rating_zscore,
            (gm.numVotes - gs.votes_mean) / gs.votes_std AS votes_zscore,
            CUME_DIST() OVER (ORDER BY gm.averageRating) AS rating_percentile,
            CUME_DIST() OVER (ORDER BY gm.numVotes) AS votes_percentile
        FROM GenreMovies AS gm, GenreStats AS gs
    )
    SELECT 
        t.primaryTitle, mr.averageRating, mr.rating_zscore,
        mr.rating_percentile, mr.numVotes, mr.votes_zscore,
        mr.votes_percentile
    FROM MovieRankings AS mr
    JOIN title_ft AS t ON mr.tconst = t.tconst
    WHERE mr.tconst = %s;

Final Optimization With Indexes

In [ ]:
%%sql

ALTER TABLE title_ft DROP INDEX idx_title_ft_avg_rating;
CREATE INDEX idx_title_ft_avg_rating ON title_ft(averageRating);

ALTER TABLE title_ft DROP INDEX idx_title_ft_num_votes;
CREATE INDEX idx_title_ft_num_votes ON title_ft(numVotes);


WITH GenreMovies AS (
        SELECT 
            ft.tconst, ft.averageRating, ft.numVotes
        FROM 
            title_ft AS ft
        JOIN 
            title_genre_bridge AS tgb ON ft.tconst = tgb.tconst
        JOIN 
            genre_dt AS g ON tgb.genreID = g.genreID
        WHERE 
            g.genreName = %s AND ft.numVotes IS NOT NULL
    ),
    GenreStats AS (
        SELECT 
            AVG(averageRating) AS rating_mean, STDDEV(averageRating) AS rating_std,
            AVG(numVotes) AS votes_mean, STDDEV(numVotes) AS votes_std
        FROM GenreMovies
    ),
    MovieRankings AS (
        SELECT
            gm.tconst, gm.averageRating, gm.numVotes,
            (gm.averageRating - gs.rating_mean) / gs.rating_std AS rating_zscore,
            (gm.numVotes - gs.votes_mean) / gs.votes_std AS votes_zscore,
            CUME_DIST() OVER (ORDER BY gm.averageRating) AS rating_percentile,
            CUME_DIST() OVER (ORDER BY gm.numVotes) AS votes_percentile
        FROM GenreMovies AS gm, GenreStats AS gs
    )
    SELECT 
        t.primaryTitle, mr.averageRating, mr.rating_zscore,
        mr.rating_percentile, mr.numVotes, mr.votes_zscore,
        mr.votes_percentile
    FROM MovieRankings AS mr
    JOIN title_ft AS t ON mr.tconst = t.tconst
    WHERE mr.tconst = %s;

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=